Check if GPU is available



In [2]:
import torch
print("CUDA available:", torch.cuda.is_available())
!nvcc --version

CUDA available: True
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


# **Installation**



There are 2 types of installing the solution. 1.) `pip install donut-python `(block below) 2.) cloning the github repo and install (block after that) 

The second option is preferable

In [3]:
!pip install donut-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 6.9 MB/s 
     |████████████████████████████████| 509 kB 60.3 MB/s 
     |████████████████████████████████| 1.3 MB 65.5 MB/s 
     |████████████████████████████████| 707 kB 65.2 MB/s 
     |████████████████████████████████| 4.9 MB 62.3 MB/s 
     |████████████████████████████████| 419 kB 72.8 MB/s 
     |████████████████████████████████| 5.9 MB 57.7 MB/s 
     |████████████████████████████████| 109 kB 67.2 MB/s 
     |████████████████████████████████| 120 kB 70.5 MB/s 
     |████████████████████████████████| 6.6 MB 55.8 MB/s 
     |████████████████████████████████| 212 kB 72.3 MB/s 
     |████████████████████████████████| 115 kB 77.7 MB/s 
     |████████████████████████████████| 127 kB 78.3 MB/s 
     |████████████████████████████████| 546 kB 75.1 MB/s 
  Created wheel for zss: filename=zss-1.2.0-py3-none-any.whl size=6747 sha256=1b721d4f81b0

In [4]:
!git clone https://github.com/clovaai/donut.git
!cd donut && pip install .

Cloning into 'donut'...
remote: Enumerating objects: 200, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 200 (delta 46), reused 36 (delta 35), pack-reused 133
Receiving objects: 100% (200/200), 62.75 MiB | 36.55 MiB/s, done.
Resolving deltas: 100% (82/82), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/donut
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for donut-python: filename=donut_python-1.0.7-py3-none-any.whl size=21235 sha256=617331c48d118c93e5aa0c652d60e40205cb6037cb6fc43dd4d

In [5]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')



Mounted at /content/gdrive


# **Inference code**



Test a cord data sample on pre-trained model

In [6]:
from donut import DonutModel
from PIL import Image
import torch
model = DonutModel.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")
if torch.cuda.is_available():
    model.half() 
    device = torch.device("cuda") 
    model.to(device) 
else: 
    model.encoder.to(torch.bfloat16)
model.eval() 
image = Image.open("/content/gdrive/My Drive/misc/sample_image_cord_test_receipt_00004.png").convert("RGB")
output = model.inference(image=image, prompt="<s_cord-v2>")
output

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/404 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/859M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536 [00:00<?, ?B/s]

ValueError: ignored

# **Training on SROIE data**

Prepare SROIE data for training. 

SROIE data should be available (extracted) in `Google Drive`. 

For this notebook; `100 training` samples, `20 validation` samples & `20 test` samples are used.

The following block prepares the data in donut trainable format. We need to change folder names (train/validation/test) and data range (see first for loop) and run the block 3 times to preapre data.

In [ ]:
import os
import json
import shutil
from tqdm.notebook import tqdm
lines = []
images = []

for ann in tqdm(os.listdir("/content/gdrive/My Drive/ICDAR-2019-SROIE/key")[120:140]):
  if ann != ".ipynb_checkpoints":
    with open("/content/gdrive/My Drive/ICDAR-2019-SROIE/key/" + ann) as f:
      data = json.load(f)
      images.append(ann[:-4] + "jpg")
      line = {"gt_parse": data}
      lines.append(line)

if not os.path.exists("/content/donut/sroie-donut/test"):
  os.makedirs("/content/donut/sroie-donut/test")

with open("/content/donut/sroie-donut/test/metadata.jsonl", 'w') as f:
  for i, gt_parse in enumerate(lines):
    line = {"file_name": images[i], "ground_truth": json.dumps(gt_parse)}
    f.write(json.dumps(line) + "\n")
    shutil.copyfile("/content/gdrive/My Drive/ICDAR-2019-SROIE/img/" + images[i], "/content/donut/sroie-donut/test/" + images[i])

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
!ls donut/sroie-donut/test

086.jpg  099.jpg  121.jpg  131.jpg  157.jpg  165.jpg  172.jpg
087.jpg  100.jpg  126.jpg  147.jpg  159.jpg  168.jpg  176.jpg
095.jpg  120.jpg  129.jpg  148.jpg  160.jpg  170.jpg  metadata.jsonl


Before commencing the training; set the hyperparams in config folder. For this perticular example, hyperparams are set in `config/train_sroie.yaml`

There are other example hyperparems files that can be edited to make this file.

below block will train the model. It will use newly created hyperparams file. The data path is configured in `config/train_sroie.yaml`

In [ ]:
!cd donut && python train.py --config config/train_sroie.yaml

resume_from_checkpoint_path: None
result_path: ./result
pretrained_model_name_or_path: naver-clova-ix/donut-base
dataset_name_or_paths: 
  - /content/donut/sroie-donut
sort_json_key: False
train_batch_sizes: 
  - 1
val_batch_sizes: 
  - 1
input_size: 
  - 1280
  - 960
max_length: 768
align_long_axis: False
num_nodes: 1
seed: 2022
lr: 3e-05
warmup_steps: 300
num_training_samples_per_epoch: 800
max_epochs: 30
max_steps: -1
num_workers: 8
val_check_interval: 1.0
check_val_every_n_epoch: 10
gradient_clip_val: 1.0
verbose: True
exp_name: train_sroie
exp_version: 20220910_132000
Config is saved at result/train_sroie/20220910_132000/config.yaml
Global seed set to 2022
/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of Do

Test an Image from SORIE data with custom trained model.

In [ ]:
from donut import DonutModel
from PIL import Image
import torch
model = DonutModel.from_pretrained("/content/donut/result/train_sroie/20220910_132000")
if torch.cuda.is_available():
    model.half()
    device = torch.device("cuda")
    model.to(device)
else:
    model.encoder.to(torch.bfloat16)
model.eval()
image = Image.open("/content/donut/sroie-donut/test/099.jpg").convert("RGB")
output = model.inference(image=image, prompt="<s_sroie-donut>")
output

/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


{'predictions': [{'company': 'SYARIKAT PERNIAGAAN GIN KEE',
   'date': '04/12/2017',
   'address': 'NO 290, JALAN AIR PANAS, SETAPAK, 53200, KUALA LUMPUR.',
   'total': '47.70'}]}

Copy entire donut folder in GDrive for downloading and future use. There will be a `result/` folder with saved model and other metrics.

In [ ]:
!cp -R /content/donut/ /content/gdrive/"My Drive"

# **Visual demo & original notebook**

In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.1 MB 34.6 MB/s 
     |████████████████████████████████| 270 kB 73.9 MB/s 
     |████████████████████████████████| 84 kB 4.1 MB/s 
     |████████████████████████████████| 2.3 MB 57.8 MB/s 
     |████████████████████████████████| 57 kB 5.5 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 212 kB 76.5 MB/s 
     |████████████████████████████████| 55 kB 3.8 MB/s 
     |████████████████████████████████| 84 kB 4.1 MB/s 
     |████████████████████████████████| 112 kB 73.0 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 80 kB 10.8 MB/s 
     |████████████████████████████████| 68 kB 7.0 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 594 kB 73.4 MB/s 
     |████████████████████████████████| 856 kB 6

In [ ]:
import argparse
import gradio as gr
import torch
from PIL import Image

from donut import DonutModel

In [ ]:
def demo_process_vqa(input_img, question):
    global pretrained_model, task_prompt, task_name
    input_img = Image.fromarray(input_img)
    user_prompt = task_prompt.replace("{user_input}", question)
    output = pretrained_model.inference(input_img, prompt=user_prompt)["predictions"][0]
    return output


def demo_process(input_img):
    global pretrained_model, task_prompt, task_name
    input_img = Image.fromarray(input_img)
    output = pretrained_model.inference(image=input_img, prompt=task_prompt)["predictions"][0]
    return output

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--task", type=str, default="cord-v2")
parser.add_argument("--pretrained_path", type=str, default="naver-clova-ix/donut-base-finetuned-cord-v2")
args, left_argv = parser.parse_known_args()

task_name = args.task
if "docvqa" == task_name:
    task_prompt = "<s_docvqa><s_question>{user_input}</s_question><s_answer>"
else:  # rvlcdip, cord, ...
    task_prompt = f"<s_{task_name}>"

pretrained_model = DonutModel.from_pretrained(args.pretrained_path)

if torch.cuda.is_available():
    pretrained_model.half()
    device = torch.device("cuda")
    pretrained_model.to(device)
else:
    pretrained_model.encoder.to(torch.bfloat16)

pretrained_model.eval()

demo = gr.Interface(
    fn=demo_process_vqa if task_name == "docvqa" else demo_process,
    inputs=["image", "text"] if task_name == "docvqa" else "image",
    outputs="json",
    title=f"Donut 🍩 demonstration for `{task_name}` task",
)
demo.launch()

/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://25264.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7f6765eae550>,
 'http://127.0.0.1:7860/',
 'https://25264.gradio.app')